In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.options.mode.chained_assignment = None  # default='warn'
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
titanic_file_path = '/kaggle/input/titanic/train.csv'
titanic_test_path = '/kaggle/input/titanic/test.csv'
titanic_data = pd.read_csv(titanic_file_path)
titanic_test = pd.read_csv(titanic_test_path)

In [3]:
print(titanic_data.columns,titanic_test.columns,sep="\n")

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


In [4]:
titanic_data = titanic_data.drop(['Cabin'],axis = 1)
titanic_test = titanic_test.drop(['Cabin'],axis = 1)

In [5]:
titanic_data.info()
# total = 891
# age = 714 (177 missing)
# cabin = missing
# embarked = 2 missing 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(4)
memory usage: 76.7+ KB


In [6]:
titanic_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(4)
memory usage: 32.8+ KB


In [7]:
titanic_data['Age'] = titanic_data['Age'].interpolate()
titanic_test['Age'] = titanic_test['Age'].interpolate()
titanic_test['Fare'] = titanic_test['Fare'].interpolate()
titanic_data = titanic_data.dropna()

In [8]:
print(titanic_data.info(),titanic_test.info(),sep="\n\n")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  889 non-null    int64  
 1   Survived     889 non-null    int64  
 2   Pclass       889 non-null    int64  
 3   Name         889 non-null    object 
 4   Sex          889 non-null    object 
 5   Age          889 non-null    float64
 6   SibSp        889 non-null    int64  
 7   Parch        889 non-null    int64  
 8   Ticket       889 non-null    object 
 9   Fare         889 non-null    float64
 10  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(4)
memory usage: 83.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name    

In [9]:
dummy = []
cols = ['Pclass', 'Sex', 'Embarked']
for i in cols:
    dummy.append(pd.get_dummies(titanic_data[i]))
titanic_dummy = pd.concat(dummy, axis=1)
titanic_data = pd.concat((titanic_data,titanic_dummy), axis=1)

dummy = []
for i in cols:
    dummy.append(pd.get_dummies(titanic_test[i]))
titanic_dummy = pd.concat(dummy, axis=1)
titanic_test = pd.concat((titanic_test,titanic_dummy), axis=1)

In [10]:
titanic_data = titanic_data.drop(cols,axis = 1)
titanic_test = titanic_test.drop(cols,axis = 1)

In [11]:
titanic_data.columns

Index(['PassengerId',    'Survived',        'Name',         'Age',
             'SibSp',       'Parch',      'Ticket',        'Fare',
                   1,             2,             3,      'female',
              'male',           'C',           'Q',           'S'],
      dtype='object')

In [12]:
titanic_test.columns

Index(['PassengerId',        'Name',         'Age',       'SibSp',
             'Parch',      'Ticket',        'Fare',             1,
                   2,             3,      'female',        'male',
                 'C',           'Q',           'S'],
      dtype='object')

In [13]:
y = titanic_data.Survived
train_X = titanic_data[['PassengerId','Age','SibSp','Parch','Fare',1,2,3,'female','male','C','Q','S']]
test_X = titanic_test[['PassengerId','Age','SibSp','Parch','Fare',1,2,3,'female','male','C','Q','S']]

In [14]:
rf_model = RandomForestClassifier(random_state = 1)
# rf_model = DecisionTreeClassifier(random_state = 1)
rf_model.fit(train_X.values,y.values)
prediction = rf_model.predict(test_X.values)
print(prediction)

[0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 0 1 0 0 1 1 1 0 1 0 1 0 0 0 0 0 1 0 0
 0 0 1 0 0 0 1 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 1 1 1 0 0 1
 1 1 0 1 0 1 1 0 0 0 0 0 1 0 1 1 0 0 1 0 1 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0
 0 1 1 1 0 0 1 1 1 1 0 1 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0
 1 0 1 0 0 1 0 0 1 0 1 1 1 1 1 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1
 0 1 0 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 1 0 1 0 0 1 0 0 0 0 0 0 0 1 0 1 0 1 0
 1 0 1 1 0 0 0 0 0 1 0 0 1 0 1 0 1 1 1 1 0 0 0 0 1 0 1 1 1 0 1 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 1 1 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 0 1 0 0 0 0 1 1 0 1 0 0 0 1 0
 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 0 1 0 0
 0 1 1 1 1 0 0 1 0 0 0]


In [15]:
test_X['Survived'] = prediction
test_X = test_X.drop(['Age','SibSp','Parch','Fare',1,2,3,'female','male','C','Q','S'],axis = 1)

In [16]:
test_X.columns

Index(['PassengerId', 'Survived'], dtype='object')

In [17]:
test_X.to_csv('submission.csv',header=True,index=False)

In [18]:
# from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay,accuracy_score
# import matplotlib.pyplot as plt

# train_X,val_X,train_y,val_y = train_test_split(train_X,y,random_state = 1)

# pred = rf_model.predict(val_X)
# print(len(pred))
# titanic_confusion_matrix = confusion_matrix(val_y,pred)
# disp = ConfusionMatrixDisplay(confusion_matrix=titanic_confusion_matrix)
# disp.plot()
# plt.show()

In [19]:
# accuracy = accuracy_score(val_y,pred)
# print(accuracy)